In [1]:
import torch
torch.cuda.is_available()

True

In [1]:
from benchmarl.algorithms import MappoConfig
from benchmarl.environments import VmasTask
from benchmarl.experiment import Experiment, ExperimentConfig
from benchmarl.models.mlp import MlpConfig

from ComplexVmasTask import ComplexVmasTask

In [3]:
!wandb enabled

W&B enabled.


In [ ]:
import gym
env = gym.make("CartPole-v1")
observation, info = env.reset(seed=42)

for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

c:\Users\Josh\Documents\yr4\s2\COMP4900_MultiAgent\COMP-4900-MARL-Project\venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [4]:
# Loads from "benchmarl/conf/experiment/base_experiment.yaml"
experiment_config = ExperimentConfig.get_from_yaml()

# Loads from "benchmarl/conf/task/vmas/balance.yaml"
# task = VmasTask.SIMPLE_SPEAKER_LISTENER.get_from_yaml()
task = ComplexVmasTask.SIMPLE_SPEAKER_LISTENER.get_from_yaml()
print(task)

# Loads from "benchmarl/conf/algorithm/mappo.yaml"
algorithm_config = MappoConfig.get_from_yaml()

# Loads from "benchmarl/conf/model/layers/mlp.yaml"
model_config = MlpConfig.get_from_yaml()
critic_model_config = MlpConfig.get_from_yaml()

# THIS IS VERY IMPORTANT !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
experiment_config.on_policy_collected_frames_per_batch = 6000
experiment_config.evaluation = True
experiment_config.render = True
# experiment_config.evaluation_interval = 12_000
experiment_config.evaluation_interval = 6_000
experiment_config.evaluation_episodes = 10

experiment_config.max_n_iters = 5
experiment_config.loggers = ["csv"]
experiment_config.create_json = True
experiment_config.save_folder = "experiments"
# experiment_config.train_device = "cuda"
# experiment_config.sampling_device = "cuda"

# Create the experiment
experiment = Experiment(
    task=task,
    algorithm_config=algorithm_config,
    model_config=model_config,
    critic_model_config=critic_model_config,
    seed=0,
    config=experiment_config,
)

ComplexVmasTask.SIMPLE_SPEAKER_LISTENER: (config={'max_steps': 100})
DEBUGGING device=cpu
DEBUGGING device=cpu


In [5]:
experiment.run()

mean return = -216.00315856933594: 100%|██████████| 5/5 [04:12<00:00, 50.51s/it]
